# Run: pmeta5 - Metagenome search

```
Lead     : ababaian
Issue    : 
Version  : v0.3.5-dev : ab-dev branch
start    : 2020 08 30
complete : 2020 08 31
files    : ~/serratus/notebook/200830_ab/
s3_files : s3://serratus-public/notebook/200830_ab/
output   : s3://serratus-public/out/200830_pmeta5/
```

### Intro/Objectives

- protref5 used as reference
- Metagenomic samples (~500K)


## Metagenome

Date: `200821`

Query: `"METAGENOMIC" NOT amplicon[All Fields] AND "platform illumina"[Properties] AND cluster_public[prop]`

Return: `473501`


Saved as: `meta_SraRunInfo.csv`

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200830_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200830_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sun Aug 30 11:30:54 PDT 2020
fde8446fbf106afff5ec1c82941f154fc39c97f7


In [2]:
# SRA Accession Initialize
cd $WORK
BATCH='meta_SraRunInfo.csv'

# Split into 2x 250k files
head -n1 $BATCH > header.tmp
split $BATCH -n 2

mv xaa meta1_SraRunInfo.csv
cat header.tmp xab > meta2_SraRunInfo.csv
rm xab

md5sum *

aws s3 sync ./ $S3_WORK

0a9601837e2e43ec3cc4472943eea408  header.tmp
5f987d45527a80591b149dfa36a69fc7  meta1_SraRunInfo.csv
871f754c41536687fc0954364533b8b3  meta2_SraRunInfo.csv
00b47c2cbd3f6c79efef7eeb7dbf135b  meta_SraRunInfo.csv
upload: ./header.tmp to s3://serratus-public/notebook/200830_ab/header.tmp
upload: ./meta2_SraRunInfo.csv to s3://serratus-public/notebook/200830_ab/meta2_SraRunInfo.csv
upload: ./meta1_SraRunInfo.csv to s3://serratus-public/notebook/200830_ab/meta1_SraRunInfo.csv
upload: ./meta_SraRunInfo.csv to s3://serratus-public/notebook/200830_ab/meta_SraRunInfo.csv


### Terraform Initialize

In [15]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve




diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..d4ed251 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@

In [16]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [5]:
cd $WORK
BATCH='meta1_SraRunInfo.csv'
wc -l $WORK/$BATCH

250861 /home/artem/serratus/notebook/200830_ab/meta1_SraRunInfo.csv


In [18]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200830_ab/meta1_SraRunInfo.csv
  date: Sun Aug 30 12:08:20 PDT 2020
  wc  : 250861 /home/artem/serratus/notebook/200830_ab/meta1_SraRunInfo.csv
  md5 : 5f987d45527a80591b149dfa36a69fc7  /home/artem/serratus/notebook/200830_ab/meta1_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
a8b23d86dbb04ab926b25b69cea74246  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
ed6bd41b3a76292041d3799ef67732bd  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
576fb701b266be5263fc770deb0fb9e2  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
69aafe8ef1cb1dd59e85cba99026ad51  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

#### Run 2

In [31]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..22972ef 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@ 

In [32]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [33]:
cd $WORK
BATCH='meta2_SraRunInfo.csv'
wc -l $WORK/$BATCH

251260 /home/artem/serratus/notebook/200830_ab/meta2_SraRunInfo.csv


In [35]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200830_ab/meta2_SraRunInfo.csv
  date: Sun Aug 30 23:15:03 PDT 2020
  wc  : 251260 /home/artem/serratus/notebook/200830_ab/meta2_SraRunInfo.csv
  md5 : 871f754c41536687fc0954364533b8b3  /home/artem/serratus/notebook/200830_ab/meta2_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
cfa8392c663bc8aaa496421d6dc482eb  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
ff9b6538bb7c1d095aa849c4a90f8fac  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
26c8654d78dae703ce3b7c2c8c5a7e45  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
53e0dd995365124b168fdc36f316647f  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

## Run Serratus

In [40]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.05,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3000,
  "CLEAR_INTERVAL": 600,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1500,
  "GENOME": "protref5",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 50,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 200,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 45
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 53  1028    0     0  100   552      0   2067 --:--:-- --:--:-- --:--:--  2067{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":25,"ALIGN_SCALING_CONSTANT":0.05,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":3000,"CLEAR_INTERVAL":600,"DL_ARGS":"","DL_MA

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
cd $TF

aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [ ]:
# Exploring sumbler data
# DO NOT UNCOMMENT WILL DELETE MANY FILES
# aws s3 sync s3://serratus-public/out/200823_test/sumbler/ ./
# find . -size -2k -exec rm '{}' \;
# grep 'SRAsum' * | sed 's/;/\t  /g' -

In [ ]:
## Done
## 248902 SRA files processed
## 250478
#
# ~27 hours at steady pace


In [ ]:
# On EC2
# Download all summaries
aws s3 sync s3://serratus-public/out/200830_pmeta5/psummary/ ./

S3DIR='s3://serratus-public/notebook/200830_ab'

function getViro {
VNAME=$1
# Grab viral family from all summary files
grep -r $VNAME \
  | grep 'seqcvg' - \
  | sort  -  \
  > ../$VNAME.pmeta5.psum
  
  aws s3 cp ../$VNAME.pmeta5.psum $S3DIR/psum/
  
}

getViro Corona
getViro Epsy
getViro Medion
getViro Roni
getViro Delta
getViro Hepe
getViro Dicistro
getViro Quenya


In [ ]:
# Manual reduce down to 'interesting cases'
cp ../$VNAME.pmeta5.psum ../$VNAME.assembly.psum
# vim $VNAME.assembly.psum

# Reduce assembly list to SRA entries only
sed 's/.*sra=//g' $VNAME.assembly.psum \
  | sed 's/;.*//g' - \
  | sort -k1 - \
  | uniq \
  > $VNAME.assembly.sra